# Assignment Chapter 2 - MACHINE LEARNING [Case #1]
Startup Campus, Indonesia - `Artificial Intelligence (AI)` (Batch 7)
* Dataset: SC_HW1_bank_data
* Libraries: Pandas, Numpy, Scikit-learn
* Objective: Classification with Supervised Machine Learning Models

`PERSYARATAN` Semua modul (termasuk versi yang sesuai) sudah di-install dengan benar.
<br>`CARA PENGERJAAN` Lengkapi baris kode yang ditandai dengan **#TODO**.
<br>`TARGET PORTFOLIO` Peserta mampu membandingkan akurasi klasifikasi dari berbagai model *supervised learning*.

### Import Library

In [ ]:
import pandas as pd
import numpy as np
np.random.seed(400)

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split as tts
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

### Read Dataset

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/Rietaros/kampus_merdeka/main/SC_HW1_bank_data.csv')
df.columns

Index(['RowNumber', 'CustomerId', 'Geography', 'Gender', 'Age', 'Tenure',
       'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember',
       'EstimatedSalary', 'Exited'],
      dtype='object')

In [ ]:
# TODO: Hilangkan kolom yang tidak relevan untuk pemodelan
df = df[['Geography', 'Gender', 'Age', 'Tenure',
       'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember',
       'EstimatedSalary', 'Exited']]

### Preprocessing

In [ ]:
# TODO: Lakukan One-Hot Encoder untuk data kategorikal, dengan fungsi pd.get_dummies
df = pd.get_dummies(df, columns=['Geography', 'Gender']).astype(int)
df

,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male
0,42,2,0,1,1,1,101348,1,1,0,0,1,0
1,41,1,83807,1,0,1,112542,0,0,0,1,1,0
2,42,8,159660,3,1,0,113931,1,1,0,0,1,0
3,39,1,0,2,0,0,93826,0,1,0,0,1,0
4,43,2,125510,1,1,1,79084,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,39,5,0,2,1,0,96270,0,1,0,0,0,1
9996,35,10,57369,1,1,1,101699,0,1,0,0,0,1
9997,36,7,0,1,0,1,42085,1,1,0,0,1,0
9998,42,3,75075,2,1,0,92888,1,0,1,0,0,1


In [ ]:
# TODO: Pisahkan fitur (X) dengan target (Y), dimana Y merujuk ke kolom "Exited"
X = df.drop('Exited', axis=1)
y = df['Exited'].values

In [ ]:
scaler = MinMaxScaler() # TODO: Lakukan scaling atau normalisasi
scaled = scaler.fit_transform(df)
X_transform = scaler.fit_transform(X)

In [ ]:
# Bagian ini digunakan jika Anda melakukan scaling atau normalisasi.
# Jika tidak, code ini bisa dilewati dan diganti dengan code yang ada di dalam comment.

X_transform = pd.DataFrame(X_transform, columns = X.columns)
# X_transform = X.copy()

### Train-Test Split

In [ ]:
X_train ,X_test, y_train, y_test = tts(X_transform, y, test_size = 0.25, random_state = 123)

### Model #1

In [ ]:
# [ PERTANYAAN ]
# TODO: Jelaskan secara singkat model #1 yang Anda gunakan.

[MODEL 1]
Untuk model 1 ini, kami menggunakan model Random Forest Classifier, model yang berbasis ensemble learning yang terdiri dari beberapa decision trees. Cara kerjanya, model ini bekerja dengan cara membuat beberapa decision tree dari subset random data training, dan result nya ditentukan oleh rerata prediksi dari semua tree(untuk regression problem) atau voting (untuk classification problem). Untuk kasus model 1 ini, kami menggunakan parameter berikut:
- n_estimators: jumlah pohon nya.
- max_depth: batas maksimum pohon, untuk membatasi pertumbuhan tree yang dihasikan.
- min_samples_split: jumlah minimum sampel yang diperlukan untuk membagi node.
- min_samples_leaf: jumlah minimum sampel pada setiap daun pohon.

In [ ]:
# CONTOH pemanggilan library dan penggunaannya:

# from sklearn.linear_model import LogisticRegression
# model1 = LogisticRegression()
# params = {"tol": [0.1,0.01,0.001], 'C':[0.5,1.0,1.5,2.0]}

# TODO: Pilih salah satu model Machine Learning (ML) dari Scikit-learn
from sklearn.ensemble import RandomForestClassifier
model1 = RandomForestClassifier()
params = {
    'n_estimators': [100, 200],
    'max_depth': [10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

# OPTIONAL: Lakukan hyperparameter tuning sesuai kebutuhan
grid = GridSearchCV(
    estimator= model1,
    param_grid= params,
    scoring = 'accuracy',
    n_jobs = 10, # core cpu yang digunakan
    cv = 10 # 3-fold cross validation (artinya kita melakukan iterasi model sebanyak 3 kali)
)

grid.fit(X_train,y_train)
grid.best_params_

{'max_depth': 10,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 200}

In [ ]:
y_pred = grid.predict(X_test) # TODO: Lakukan evaluasi untuk model #1

print(classification_report(y_test,y_pred))
print("confusion matrix :")
print(confusion_matrix(y_test,y_pred),"\n")
print("accuracy score : ")
print(accuracy_score(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.87      0.97      0.92      1983
           1       0.79      0.45      0.57       517

    accuracy                           0.86      2500
   macro avg       0.83      0.71      0.75      2500
weighted avg       0.86      0.86      0.85      2500

confusion matrix :
[[1923   60]
 [ 285  232]] 

accuracy score : 
0.862


### Model #2

In [ ]:
# [ PERTANYAAN ]
# TODO: Jelaskan secara singkat model #2 yang Anda gunakan.

[MODEL 2]
Untuk model 2 ini, kami menggunakan model Logistic Regression, model yang digunakan untuk klasifikasi biner (true or false). Dengan menggunakan fungsi logistik (sigmoid), model ini memprediksi probabilitas suatu class berdasarkan variabel input. Model ini mudah diinterpretasikan, serta cepat untuk dieksekusi, cocok untuk dataset yang memiliki hubungan linier antara fitur. Untuk kasus model 2 ini, kami menggunakan parameter berikut:
- tol: batas toleransi untuk menghentikan iterasi.
- C: parameter regularisasi untuk mengontrol overfitting.
- max_iter: jumlah maksimum iterasi dalam proses pelatihan.

In [ ]:
# CONTOH pemanggilan library dan penggunaannya:

from sklearn.linear_model import LogisticRegression
# model1 = LogisticRegression()
# params = {"tol": [0.1,0.01,0.001], 'C':[0.5,1.0,1.5,2.0]}


# TODO: Pilih salah satu model Machine Learning (ML) dari Scikit-learn
model2 = LogisticRegression()
params = {
    'tol': [0.1, 0.01, 0.001],
    'C': [0.5, 1.0, 1.5, 2.0],
    'max_iter': [100, 200]
}
# OPTIONAL: Lakukan hyperparameter tuning sesuai kebutuhan
grid = GridSearchCV(
    estimator= model2,
    param_grid= params,
    scoring = 'accuracy',
    n_jobs = 10, # core cpu yang digunakan
    cv = 10 # 3-fold cross validation (artinya kita melakukan iterasi model sebanyak 3 kali)
)

grid.fit(X_train,y_train)
grid.best_params_

{'C': 0.5, 'max_iter': 100, 'tol': 0.001}

In [ ]:
y_pred = grid.predict(X_test) # TODO: Lakukan evaluasi untuk model #2

print("Logistic Regression Classification Report:")
print(classification_report(y_test,y_pred))
print("Confusion Matrix:")
print(confusion_matrix(y_test,y_pred))
print("Accuracy Score:")
print(accuracy_score(y_test, y_pred))

Logistic Regression Classification Report:
              precision    recall  f1-score   support

           0       0.82      0.97      0.89      1983
           1       0.66      0.20      0.31       517

    accuracy                           0.81      2500
   macro avg       0.74      0.59      0.60      2500
weighted avg       0.79      0.81      0.77      2500

Confusion Matrix:
[[1930   53]
 [ 413  104]]
Accuracy Score:
0.8136


### Model #3

In [ ]:
# [ PERTANYAAN ]
# TODO: Jelaskan secara singkat model #3 yang Anda gunakan.

[MODEL 3] Untuk model 3 ini, kami menggunakan algortima KNN, algoritma non-parametrik yang digunakan untuk klasifikasi. Cara kerja model ini,kita menghitung jarak antara titik data baru dan data yang udah dilatih, lalu diprediksi kelas berdasarkan mayoritas dari k node terdekat. Untuk kasus model 3 ini, kami menggunakan parameter berikut:
- n_neighbors: jumlah tetangga yang dipertimbangkan.
- weights: cara menghitung bobot tetangga (uniform atau distance).
- metric: metrik jarak yang digunakan (euclidean atau manhattan).

In [ ]:
# CONTOH pemanggilan library dan penggunaannya:

# from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
# model1 = LogisticRegression()
# params = {"tol": [0.1,0.01,0.001], 'C':[0.5,1.0,1.5,2.0]}


# TODO: Pilih salah satu model Machine Learning (ML) dari Scikit-learn
model3 = KNeighborsClassifier()
params = {
    'n_neighbors': [3, 5, 7],  # Tetapkan beberapa nilai untuk tetangga
    'weights': ['uniform', 'distance'],  # Coba bobot 'uniform' atau 'distance'
    'metric': ['euclidean', 'manhattan']  # Pilih antara metrik jarak Euclidean atau Manhattan
}

# OPTIONAL: Lakukan hyperparameter tuning sesuai kebutuhan
grid = GridSearchCV(
    estimator= model3,
    param_grid= params,
    scoring = 'accuracy',
    n_jobs = 10, # core cpu yang digunakan
    cv = 10 # 3-fold cross validation (artinya kita melakukan iterasi model sebanyak 3 kali)
)

grid.fit(X_train,y_train)
grid.best_params_

{'metric': 'manhattan', 'n_neighbors': 7, 'weights': 'distance'}

In [ ]:
y_pred = grid.predict(X_test) # TODO: Lakukan evaluasi untuk model #3

print("KNN Classification Report:")
print(classification_report(y_test,y_pred))
print("Confusion Matrix:")
print(confusion_matrix(y_test,y_pred))
print("Accuracy Score:")
print(accuracy_score(y_test, y_pred))

KNN Classification Report:
              precision    recall  f1-score   support

           0       0.85      0.95      0.89      1983
           1       0.63      0.34      0.45       517

    accuracy                           0.82      2500
   macro avg       0.74      0.65      0.67      2500
weighted avg       0.80      0.82      0.80      2500

Confusion Matrix:
[[1879  104]
 [ 339  178]]
Accuracy Score:
0.8228


### Conclusion

In [ ]:
# [ PERTANYAAN ]
# TODO: Tarik kesimpulan model mana yang terbaik beserta alasannya

Dilihat dari hasil evaluasi model menggunakan metrik, yaitu seperti skor akurasi, classification report (precision, recall, F1-score), dan confusion matrix nya. Maka, dapat ditarik kesimpulan bahwa Model 1 yakni Random Forest Classifier adalah algoritma yang terbaik pada kasus Memprediksi Churn Status di Perbankan ini dengan nilai akurasi = 0.8604. Model ini memiliki kelebihan dapat menangani kompleksitas data dengan baik dan unggul dalam situasi dimana ada banyak fitur yang berinteraksi secara non-linier, misal dalam kasus prediksi churn status pelanggan diperbankan ini. Random Forest mampu membuat beberapa decision trees berdasarkan subset random dari data, yang membuatnya lebih robust terhadap noise dan overfitting daripada Logistic Regression atau KNN.

### Scoring
Total `#TODO` = 13
<br>Checklist:

- [✔️] Hilangkan kolom yang tidak relevan untuk pemodelan
- [✔️] Lakukan One-Hot Encoder untuk data kategorikal, dengan fungsi pd.get_dummies
- [✔️] Pisahkan fitur (X) dengan target (Y), dimana Y merujuk ke kolom "Exited"
- [✔️] Lakukan scaling atau normalisasi
- [✔️] Jelaskan secara singkat model #1 yang Anda gunakan
- [✔️] Pilih salah satu model Machine Learning (ML) dari Scikit-learn (model #1)
- [✔️] Lakukan evaluasi untuk model #1
- [✔️] Jelaskan secara singkat model #2 yang Anda gunakan
- [✔️] Pilih salah satu model Machine Learning (ML) dari Scikit-learn (model #2)
- [✔️] Lakukan evaluasi untuk model #2
- [✔️] Jelaskan secara singkat model #3 yang Anda gunakan
- [✔️] Pilih salah satu model Machine Learning (ML) dari Scikit-learn (model #3)
- [✔️] Lakukan evaluasi untuk model #3

### Additional readings
- N/A

### Copyright © 2024 Startup Campus, Indonesia
* You may **NOT** use this file except there is written permission from PT. Kampus Merdeka Belajar (Startup Campus).
* Please address your questions to mentors.